In [1]:
import pandas as pd
import numpy as np

### Predict week 12
Need: 
- game stats from week 11
    - home team
    - away team
- prev game averages, last 5 games
    - home team
    - away team
- tweet stats leading up to week 12
    - home team
    - away team
- 

game_stats/wk_12_sentstats.csv 
- tweet stats for home and away
- score, spread 
- week, teams

final_stats/final_season_team_stats.csv
- prev game stats 
Game Stats for week 12 (week 11) = game_stats/final_season_team_stats.csv
- each row is individual team and week so would look up
- set index to team then week

In [16]:
team_stats = pd.read_csv('./final_stats/final_season_team_stats.csv', index_col=0)
team_stats = team_stats.set_index(['team','week'])
team_stats = team_stats.drop(['wk'], axis=1)
team_stats = team_stats.fillna(0)


In [3]:
def get_team_wk(team, wk, df=team_stats):
    return df.xs((team,wk), level=[0,1])

def get_team(team, df=team_stats):
    return df.loc[[team]]

In [4]:
# get_team_wk(team_stats, 'bears', 15)
bears = get_team('bears')
bears.columns
# wk, record, timeposs?, 

Index(['day', 'date', 'home/away', 'opponent', 'score_team', 'score_opp',
       'pass_cmp_off', 'pass_att_off', 'pass_yds_off', 'pass_tds_off',
       'ints_off', 'sacks_off', 'sacks_yds_off', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_prcnt', 'pass_qb_rate',
       'rush_att_off', 'rush_yds_off', 'rush_yds_per_att', 'rush_tds_off',
       'fg_made', 'fg_att', '3d_made', '3d_att', '4d_made', '4d_att',
       'time_poss', 'record', '1st_down_off', 'turnover_off', '1st_down_def',
       'tot_yds_def', 'pass_yds_def', 'rush_yds_def', 'turnover_def',
       'exp_off', 'exp_def', 'exp_st', 'spread', 'def_rush_att',
       'def_rush_tds', 'off_rush_att', 'off_rush_tds', 'def_pass_cmp',
       'def_pass_att', 'def_pass_tds', 'def_pass_ints', 'off_pass_cmp',
       'off_pass_att', 'off_pass_tds', 'off_pass_ints', 'def_sacks',
       'def_sack_yds', 'tm_penalty', 'tm_penalty_yds', 'opp_penalty',
       'opp_penalty_yds'],
      dtype='object')

In [6]:
def get_rename_dict(n):
    return {
    'score_team': f'score_team_last_{n}',
    'score_opp': f'score_opp_last_{n}',
    'pass_cmp_off': f'pass_cmp_off_last_{n}',
    'pass_att_off': f'pass_att_off_last_{n}',
    'pass_yds_off': f'pass_yds_off_last_{n}',
    'pass_tds_off': f'pass_tds_off_last_{n}',
    'ints_off': f'ints_off_last_{n}',
    'sacks_off': f'sacks_off_last_{n}',
    'sacks_yds_off': f'sacks_yds_off_last_{n}',
    'pass_yds_per_att': f'pass_yds_per_att_last_{n}',
    'pass_net_yds_per_att': f'pass_net_yds_per_att_last_{n}',
    'pass_cmp_prcnt': f'pass_cmp_prcnt_last_{n}',
    'pass_qb_rate': f'pass_qb_rate_last_{n}',
    'rush_att_off': f'rush_att_off_last_{n}',
    'rush_yds_off': f'rush_yds_off_last_{n}',
    'rush_yds_per_att': f'rush_yds_per_att_last_{n}',
    'rush_tds_off': f'rush_tds_off_last_{n}',
    'fg_made': f'fg_made_last_{n}',
    'fg_att': f'fg_att_last_{n}',
    '3d_made': f'3d_made_last_{n}',
    '3d_att': f'3d_att_last_{n}',
    '4d_made': f'4d_made_last_{n}',
    '4d_att': f'4d_att_last_{n}',
    '1st_down_off': f'1st_down_off_last_{n}',
    'turnover_off': f'turnover_off_last_{n}',
    '1st_down_def': f'1st_down_def_last_{n}',
    'tot_yds_def': f'tot_yds_def_last_{n}',
    'pass_yds_def': f'pass_yds_def_last_{n}',
    'rush_yds_def': f'rush_yds_def_last_{n}',
    'turnover_def': f'turnover_def_last_{n}',
    'exp_off': f'exp_off_last_{n}',
    'exp_def': f'exp_def_last_{n}',
    'exp_st': f'exp_st_last_{n}',
    'spread': f'spread_last_{n}',
    'def_rush_att': f'def_rush_att_last_{n}',
    'def_rush_tds': f'def_rush_tds_last_{n}',
    'off_rush_att': f'off_rush_att_last_{n}',
    'off_rush_tds': f'off_rush_tds_last_{n}',
    'def_pass_cmp': f'def_pass_cmp_last_{n}',
    'def_pass_att': f'def_pass_att_last_{n}',
    'def_pass_tds': f'def_pass_tds_last_{n}',
    'def_pass_ints': f'def_pass_ints_last_{n}',
    'off_pass_cmp': f'off_pass_cmp_last_{n}',
    'off_pass_att': f'off_pass_att_last_{n}',
    'off_pass_tds': f'off_pass_tds_last_{n}',
    'off_pass_ints': f'off_pass_ints_last_{n}',
    'def_sacks': f'def_sacks_last_{n}',
    'def_sack_yds': f'def_sack_yds_last_{n}',
    'tm_penalty': f'tm_penalty_last_{n}',
    'tm_penalty_yds': f'tm_penalty_yds_last_{n}',
    'opp_penalty': f'opp_penalty_last_{n}',
    'opp_penalty_yds': f'opp_penalty_yds_last_{n}'
}

In [127]:
def get_ave_values(df):
    return df.mean().to_dict()

# figure out how to get select rows based on n game parevious
def get_prev_n_mean(df, wk, n):
    # get team name
    team = df.index.get_level_values(0).to_list()[0]
    # get all weeks team played for indices
    wk_idx = df.index.get_level_values(1).to_list()
    
    # get index value for the last week
    if wk not in wk_idx:
        print(f'no game {wk}')
        return 
    end_idx = wk_idx.index(wk)
    
    # get start wk by subtracting n from end wk index value
    start_wk = wk_idx[end_idx-n]
    # print('start_wk', start_wk,  'end_wk', wk-1)
    # slice last n rows
    df_last_n = df.loc[(team, start_wk):(team, wk-1)]

    # get average values and put in dataframe
    ave_values_df = pd.DataFrame.from_dict([get_ave_values(df_last_n)])
    
    # set week and team columns and use and indices
    ave_values_df[['team','week']] = [team,wk]
    ave_values_df = ave_values_df.set_index(['team','week'])

    # rename columns
    rename_dict = get_rename_dict(n)
    ave_values_df = ave_values_df.rename(columns=rename_dict)

    return ave_values_df

# wk 15, get last game
# start is 13, end is 15 or 14?

ave_values = get_prev_n_mean(bears, 13, 1)
ave_values

,,score_team_last_1,score_opp_last_1,pass_cmp_off_last_1,pass_att_off_last_1,pass_yds_off_last_1,pass_tds_off_last_1,ints_off_last_1,sacks_off_last_1,sacks_yds_off_last_1,pass_yds_per_att_last_1,...,off_pass_cmp_last_1,off_pass_att_last_1,off_pass_tds_last_1,off_pass_ints_last_1,def_sacks_last_1,def_sack_yds_last_1,tm_penalty_last_1,tm_penalty_yds_last_1,opp_penalty_last_1,opp_penalty_yds_last_1
team,week,,,,,,,,,,,,,,,,,,,,,
bears,13,10.0,31.0,14.0,25.0,165.0,1.0,1.0,2.0,14.0,7.2,...,14.0,25.0,1.0,1.0,1.0,7.0,1.0,10.0,1.0,5.0


In [128]:
bears.loc[('bears', 13):('bears', 14)]

,,day,date,home/away,opponent,score_team,score_opp,pass_cmp_off,pass_att_off,pass_yds_off,pass_tds_off,...,off_pass_cmp,off_pass_att,off_pass_tds,off_pass_ints,def_sacks,def_sack_yds,tm_penalty,tm_penalty_yds,opp_penalty,opp_penalty_yds
team,week,,,,,,,,,,,,,,,,,,,,,
bears,13,Sun,December 4,home,packers,19,28,20,25,254,0,...,20,25,0,2,0,0,5,63,2,20


### Get average last 5 game stats



In [129]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [130]:
bears.xs(('bears',12),)['spread']

-7.0

In [131]:
# for game in wk 18, get last n game stats (not including 18)
def get_spread(df, wk):
    team = df.index.get_level_values(0).to_list()[0]
    return df.xs((team, wk),)['spread']
    

def get_average_last_n_games(team_df, wks, n):
    values = []
    for wk in wks:
        wk_idx = team_df.index.get_level_values(1).to_list()
        if wk not in wk_idx:
            print(f'no week {wk}')
        else: 
            # get last n games averaged
            spread = get_spread(team_df, wk)
            # print(spread)
            last_n = get_prev_n_mean(team_df, wk, n)
            # get last game stats
            last_1 = get_prev_n_mean(team_df, wk, 1)
            new_df = pd.concat([last_1, last_n], axis=1)
            new_df['spread'] = spread
            # get twitter stats
            values.append(new_df)
    stats_df = pd.concat(values)
    return stats_df
    
wk_range = list(range(12,20))

team_stats_df = get_average_last_n_games(bears, wk_range, 5)
team_stats_df
# get_spread(bears, 12)

no week 14
no week 19


score_team_last_1  score_opp_last_1  pass_cmp_off_last_1  \
team  week                                                             
bears 12                 24.0              27.0                 14.0   
      13                 10.0              31.0                 14.0   
      15                 19.0              28.0                 20.0   
      16                 20.0              25.0                 14.0   
      17                 13.0              35.0                 18.0   
      18                 10.0              41.0                  7.0   

            pass_att_off_last_1  pass_yds_off_last_1  pass_tds_off_last_1  \
team  week                                                                  
bears 12                   21.0                128.0                  1.0   
      13                   25.0                165.0                  1.0   
      15                   25.0                254.0                  0.0   
      16                   22.0                 91.0                  2.0   
      17                   28.0                129.0                  1.0   
      18                   21.0                 30.0                  1.0   

            ints_off_last_1  sacks_off_last_1  sacks_yds_off_last_1  \
team  week                                                            
bears 12                1.0               4.0                  25.0   
      13                1.0               2.0                  14.0   
      15                2.0               0.0                   0.0   
      16                0.0               6.0                  61.0   
      17                1.0               2.0                  15.0   
      18                1.0               7.0                  45.0   

            pass_yds_per_att_last_1  ...  off_pass_att_last_5  \
team  week                           ...                        
bears 12                        7.3  ...                 22.8   
      13                        7.2  ...                 23.6   
      15                       10.2  ...                 23.8   
      16                        6.9  ...                 22.6   
      17                        5.1  ...                 24.2   
      18                        3.6  ...                 24.2   

            off_pass_tds_last_5  off_pass_ints_last_5  def_sacks_last_5  \
team  week                                                                
bears 12                    1.8                   0.6               0.8   
      13                    1.8                   0.6               0.8   
      15                    1.4                   1.0               0.6   
      16                    1.2                   1.0               0.8   
      17                    1.0                   1.0               0.6   
      18                    1.0                   1.0               1.0   

            def_sack_yds_last_5  tm_penalty_last_5  tm_penalty_yds_last_5  \
team  week                                                                  
bears 12                    4.8                6.0                   62.8   
      13                    4.6                5.4                   57.8   
      15                    3.0                5.2                   61.4   
      16                    4.2                5.6                   49.0   
      17                    2.6                4.2                   34.8   
      18                    5.8                3.4                   33.4   

            opp_penalty_last_5  opp_penalty_yds_last_5  spread  
team  week                                                      
bears 12                   3.4                    27.4    -7.0  
      13                   2.8                    22.4     4.5  
      15                   2.0                    18.2     8.5  
      16                   1.8                    15.6     8.0  
      17                   3.2                    27.0    -6.0  
      18                   4.0                    32.8 

In [132]:
test_teams = ['bears','49ers']
teams = team_stats.index.unique(0).to_list()

def get_all_teams(teams, df, n):
    dfs = []
    wk_range = list(range(12,20))
    for team in teams:
        team_df = get_team(team, df)
        team_df_stats = get_average_last_n_games(team_df, wk_range, n)
        dfs.append(team_df_stats)
    all_df = pd.concat(dfs)
    return all_df

all_team_stats_7 = get_all_teams(teams, team_stats, 7)

# all_team_stats_7
        

no week 14
no week 19
no week 14
no week 19
no week 13
no week 19
no week 19
no week 14
no week 19
no week 17
no week 19
no week 19
no week 14
no week 19
no week 19
no week 19
no week 14
no week 19
no week 19
no week 19
no week 13
no week 19
no week 19
no week 14
no week 19
no week 19
no week 19
no week 19
no week 19
no week 17


In [119]:
# all_team_stats_7.to_csv('./final_stats/final_game_stats_last_7.csv')

sentiment alreday atached to game vs stats so need to add team game stats   
save all the game stats back into a df

In [120]:
all_team_stats_7.xs(('bengals', 12), level=[0,1])

,,score_team_last_1,score_opp_last_1,pass_cmp_off_last_1,pass_att_off_last_1,pass_yds_off_last_1,pass_tds_off_last_1,ints_off_last_1,sacks_off_last_1,sacks_yds_off_last_1,pass_yds_per_att_last_1,...,off_pass_att_last_7,off_pass_tds_last_7,off_pass_ints_last_7,def_sacks_last_7,def_sack_yds_last_7,tm_penalty_last_7,tm_penalty_yds_last_7,opp_penalty_last_7,opp_penalty_yds_last_7,spread
team,week,,,,,,,,,,,,,,,,,,,,,
bengals,12,37.0,30.0,24.0,39.0,346.0,4.0,2.0,2.0,9.0,9.1,...,35.857143,2.285714,0.571429,1.428571,9.142857,4.428571,32.285714,5.714286,45.857143,-1.5


## start from here
working on finalizing final stats with sentiment, function works need to run for each week


In [168]:
all_team_stats_3 = pd.read_csv('./final_stats/final_game_stats_last_3.csv', index_col=['team','week'])
all_team_stats_5 = pd.read_csv('./final_stats/final_game_stats_last_5.csv', index_col=['team','week'])
all_team_stats_7 = pd.read_csv('./final_stats/final_game_stats_last_7.csv', index_col=['team','week'])
all_team_stats_10 = pd.read_csv('./final_stats/final_game_stats_last_10.csv', index_col=['team','week'])

In [167]:
all_team_stats_3

score_team_last_1  score_opp_last_1  pass_cmp_off_last_1  \
team    week                                                             
falcons 12                 27.0              24.0                 13.0   
        13                 13.0              19.0                 15.0   
        15                 16.0              19.0                 13.0   
        16                 18.0              21.0                 13.0   
        17                  9.0              17.0                 22.0   
...                         ...               ...                  ...   
bills   14                 24.0              10.0                 22.0   
        15                 20.0              12.0                 16.0   
        16                 32.0              29.0                 25.0   
        18                 35.0              13.0                 15.0   
        19                 35.0              23.0                 19.0   

              pass_att_off_last_1  pass_yds_off_last_1  pass_tds_off_last_1  \
team    week                                                                  
falcons 12                   20.0                131.0                  1.0   
        13                   25.0                165.0                  1.0   
        15                   24.0                160.0                  1.0   
        16                   26.0                 89.0                  0.0   
        17                   33.0                212.0                  0.0   
...                           ...                  ...                  ...   
bills   14                   33.0                223.0                  2.0   
        15                   27.0                130.0                  1.0   
        16                   40.0                296.0                  4.0   
        18                   26.0                172.0                  2.0   
        19                   31.0                237.0                  3.0   

              ints_off_last_1  sacks_off_last_1  sacks_yds_off_last_1  \
team    week                                                            
falcons 12                0.0               0.0                   0.0   
        13                1.0               1.0                   9.0   
        15                1.0               1.0                   7.0   
        16                0.0               4.0                   8.0   
        17                0.0               2.0                   6.0   
...                       ...               ...                   ...   
bills   14                0.0               2.0                   0.0   
        15                0.0               3.0                  17.0   
        16                0.0               2.0                   8.0   
        18                2.0               1.0                   0.0   
        19                1.0               2.0                  17.0   

              pass_yds_per_att_last_1  ...  off_pass_att_last_3  \
team    week                           ...                        
falcons 12                        6.6  ...            24.333333   
        13                        7.0  ...            25.000000   
        15                        7.0  ...            23.000000   
        16                        3.7  ...            25.000000   
        17                        6.6  ...            27.666667   
...                               ...  ...                  ...   
bills   14                        6.8  ...            34.000000   
        15                        5.4  ...            34.000000   
        16                        7.6  ...            33.333333   
        18                        6.6  ...            31.000000   
        19                        8.2  ...            32.333333   

              off_pass_tds_last_3  off_pass_ints_last_3  def_sacks_last_3  \
team    week                                                                
falcons 12               1.000000              

In [152]:
weeks = list(range(12,20,1))
weeks

[12, 13, 14, 15, 16, 17, 18, 19]

In [159]:
wk_df_12 = pd.read_csv('./game_stats/wk_12_sentstats.csv', index_col=[0])
wk_df_12 = wk_df_12.iloc[3:].drop([6])
# wk_df_13 = pd.read_csv('./game_stats/wk_13_sentstats.csv', index_col=[0])
# wk_df_13
wk_df_12

,Week,Day,Date,home,away,home_score,away_score,away_positive96,away_positive24,away_neutral96,...,away_swing,home_positive96,home_positive24,home_neutral96,home_neutral24,home_negative96,home_negative24,home_sentiment_96h,home_sentiment_24h,home_swing
3,12,Sun,2022-11-27,titans,bengals,16,20,885.0,340.0,1098.0,...,-0.010565,472.0,187.0,865.0,286.0,147.0,56.0,0.219003,0.247637,0.028634
4,12,Sun,2022-11-27,browns,buccaneers,23,17,469.0,186.0,701.0,...,0.021515,878.0,356.0,1315.0,527.0,423.0,195.0,0.173930,0.149351,-0.024579
5,12,Sun,2022-11-27,dolphins,texans,30,15,390.0,129.0,753.0,...,0.056979,2795.0,1019.0,1130.0,365.0,277.0,66.0,0.599238,0.657241,0.058003
7,12,Sun,2022-11-27,commanders,falcons,19,13,239.0,80.0,600.0,...,-0.014771,609.0,262.0,766.0,311.0,185.0,82.0,0.271795,0.274809,0.003014
8,12,Sun,2022-11-27,panthers,broncos,23,10,671.0,197.0,804.0,...,-0.010831,114.0,38.0,238.0,58.0,32.0,9.0,0.213542,0.276190,0.062649
9,12,Sun,2022-11-27,jets,bears,31,10,563.0,204.0,1531.0,...,0.006403,908.0,295.0,1967.0,538.0,498.0,118.0,0.121554,0.186120,0.064566
10,12,Sun,2022-11-27,seahawks,raiders,34,40,1031.0,337.0,1318.0,...,-0.029332,373.0,157.0,618.0,226.0,41.0,16.0,0.321705,0.353383,0.031678
11,12,Sun,2022-11-27,cardinals,chargers,24,25,417.0,137.0,580.0,...,0.040073,61.0,27.0,238.0,67.0,16.0,1.0,0.142857,0.273684,0.130827
12,12,Sun,2022-11-27,chiefs,rams,26,10,236.0,80.0,862.0,...,0.019397,1184.0,488.0,1216.0,416.0,172.0,59.0,0.393468,0.445483,0.052015
13,12,Sun,2022-11-27,49ers,saints,13,0,287.0,126.0,808.0,...,0.063280,1165.0,449.0,1368.0,445.0,216.0,90.0,0.345216,0.364837,0.019621


In [ ]:
all_team_stats_3

In [175]:

def get_home_dict(columns):
    home_dict = {}
    for col in columns:
        home_dict[col] = f'home_{col}'
    return home_dict

def get_away_dict(columns):
    away_dict = {}
    for col in columns:
        away_dict[col] = f'away_{col}'
    return away_dict

# wk_df_12 = pd.read_csv('./game_stats/wk_12_sentstats.csv', index_col=[0])
# wk_df_12 = wk_df_12.iloc[3:].drop([6])

def get_all_records(wk_df, all_team_stats, n_games):
    all_records = []
    idxs = []

    for row in wk_df.itertuples():
        home_tm = row.home
        away_tm = row.away
        wk = row.Week

        print(f'Week {wk} {away_tm} @ {home_tm}')
        # get home and away stats from the all team stats df
        home_stats_df = all_team_stats.xs((home_tm,wk), level=[0,1])
        away_stats_df = all_team_stats.xs((away_tm,wk), level=[0,1])
        print('got Here')

        # create a column rename dictionary from current columns for home and away
        home_col_dict = get_home_dict(home_stats_df.columns)    
        away_col_dict = get_away_dict(away_stats_df.columns)

        # rename columns
        home_stats_df = home_stats_df.rename(columns = home_col_dict)
        away_stats_df = away_stats_df.rename(columns = away_col_dict)
        # convert rows to records dict
        home_records = home_stats_df.to_dict('records')
        away_records = away_stats_df.to_dict('records')

        # combine the dictionary into one dict and append to list
        records = {**home_records[0], **away_records[0]}
        all_records.append(records)
        # get index for row and append to list as well
        idxs.append(row.Index)

    # create dataframe from all records
    all_records_df = pd.DataFrame.from_dict(all_records)
    # set index from list of indexs so they match records
    all_records_df = all_records_df.set_index([pd.Index(idxs)])
    # concat wk df and all records df
    new_wk = pd.concat([wk_df, all_records_df], axis = 1)
    new_wk.to_csv(f'./final_stats/combo_sent_game/{n_games}_games/final_sent_w_game_wk_{wk}_games_{n_games}.csv')
    return new_wk



In [204]:
n_game_list = ['3','5','7','10']
all_team_stats_dict = {
    '3': all_team_stats_3,
    '5': all_team_stats_5,
    '7': all_team_stats_7,
    '10': all_team_stats_10,
}

def get_all_stats(n_game_list, all_team_stats_dict):
    for week in ['15']:
        wk_df = pd.read_csv(f'./game_stats/wk_{str(week)}_sentstats.csv', index_col=[0])
        for n in n_game_list:
            all_team_stats = all_team_stats_dict[n]
            get_all_records(wk_df, all_team_stats, n)



In [205]:
get_all_stats(n_game_list, all_team_stats_dict)

Week 15 49ers @ seahawks
got Here
Week 15 colts @ vikings
got Here
Week 15 ravens @ browns
got Here
Week 15 dolphins @ bills
got Here
Week 15 falcons @ saints
got Here
Week 15 steelers @ panthers
got Here
Week 15 eagles @ bears
got Here
Week 15 cowboys @ jaguars
got Here
Week 15 lions @ jets
got Here
Week 15 chiefs @ texans
got Here
Week 15 giants @ commanders
got Here
Week 15 cardinals @ broncos
got Here
Week 15 bengals @ buccaneers
got Here
Week 15 49ers @ seahawks
got Here
Week 15 colts @ vikings
got Here
Week 15 ravens @ browns
got Here
Week 15 dolphins @ bills
got Here
Week 15 falcons @ saints
got Here
Week 15 steelers @ panthers
got Here
Week 15 eagles @ bears
got Here
Week 15 cowboys @ jaguars
got Here
Week 15 lions @ jets
got Here
Week 15 chiefs @ texans
got Here
Week 15 giants @ commanders
got Here
Week 15 cardinals @ broncos
got Here
Week 15 bengals @ buccaneers
got Here
Week 15 49ers @ seahawks
got Here
Week 15 colts @ vikings
got Here
Week 15 ravens @ browns
got Here
Week 1

In [206]:
import os
for n in ['3','5','7','10']:
    print(n)
    files = os.listdir(f'./final_stats/combo_sent_game/{n}_games/')
    combined_df = pd.concat([pd.read_csv(f'./final_stats/combo_sent_game/{n}_games/{f}') for f in files])
    combined_df = combined_df.drop(['Unnamed: 0'], axis=1)
    combined_df.to_csv(f'./final_stats/final_season_sent_game_last_{n}.csv')

3
5
7
10


In [211]:
df = pd.read_csv('./final_stats/final_season_sent_game_last_10.csv', index_col=[0])
df.isnull().values.any()

False